#### 라이브러리 불러오기

In [65]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tensor import dataset_spec
from tensor import make_window_from_array

In [66]:
_PATH_BASE = os.path.join(
    os.path.expanduser('~'), 
    'projects', 'ai-projects', '61-WIND-TURBIN', 'data'
)
df_data = pd.read_csv(os.path.join(_PATH_BASE, 'b_turbin.csv'))

In [67]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22326 entries, 0 to 22325
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TURBINE_TIME  22326 non-null  object 
 1   WIND_SPEED    22326 non-null  float64
 2   WIND_DIR      22326 non-null  float64
 3   ACTIVE_POWER  22326 non-null  float64
 4   month         22326 non-null  int64  
 5   hour          22326 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 1.0+ MB


In [68]:
df_data['TURBINE_TIME'] = pd.to_datetime(df_data['TURBINE_TIME'])
df_data.set_index('TURBINE_TIME', inplace=True)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22326 entries, 2020-01-15 22:00:00 to 2022-08-03 03:00:00
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   WIND_SPEED    22326 non-null  float64
 1   WIND_DIR      22326 non-null  float64
 2   ACTIVE_POWER  22326 non-null  float64
 3   month         22326 non-null  int64  
 4   hour          22326 non-null  int64  
dtypes: float64(3), int64(2)
memory usage: 1.0 MB


In [69]:
df_data.head()

,WIND_SPEED,WIND_DIR,ACTIVE_POWER,month,hour
TURBINE_TIME,,,,,
2020-01-15 22:00:00,3.350000,-7.000000,91.579167,1,22
2020-01-15 23:00:00,4.044776,1.011940,215.079104,1,23
2020-01-16 00:00:00,4.355769,-1.425000,273.946154,1,0
2020-01-16 01:00:00,4.041667,-0.678333,214.488333,1,1
2020-01-16 02:00:00,3.755932,1.927119,163.486441,1,2


In [70]:
WINDOW_SIZE = 10
COLUMN_INDICES = {name: i for i, name in enumerate(df_data.columns)}
TARGET_COL = 'ACTIVE_POWER'
TARGET_COL_IDX = COLUMN_INDICES[TARGET_COL]
BATCH_SIZE = 10

In [71]:
nd_data = df_data.values

In [72]:
# def make_window(data, window_size, batch_size, shuffle=False):
#     ds = tf.data.Dataset.from_tensor_slices(data)
#     ds = ds.window(window_size+1, shift=1, drop_remainder=True)
#     ds = ds.flat_map(lambda w: w.batch(window_size+1))
#     if shuffle:
#         ds = ds.shuffle(1000)
#     ds = ds.map(lambda w: (w[:-1], w[-1, TARGET_COL_IDX]))
#     return ds.batch(batch_size).prefetch(1)

In [73]:
ts_train = make_window_from_array(
    nd_data, WINDOW_SIZE, BATCH_SIZE, TARGET_COL_IDX,
    # drop_label=True,
    last_idx_label=True,
)

In [74]:
dataset_spec(ts_train)

Data shape: (10, 10, 4), (10,), 

Last data shape: (7, 10, 4), (7,)
Batch data size: 2232, Total data size: 22317


In [75]:
is_match = True
idx = 0
x_data = np.delete(nd_data, TARGET_COL_IDX, axis=1)
# x_data = nd_data
y_data = nd_data[WINDOW_SIZE-1:, TARGET_COL_IDX]
for x, y in ts_train.take(-1):
    idxx1 = 0
    for xe in x:
        idxx2 = 0
        for xee in xe.numpy():
            if np.all(x_data[idx*WINDOW_SIZE+idxx1+idxx2] != xee):
                print(f'Not match X: \n{x_data[idx+idxx1+idxx2]}\n{xee}')
                is_match = False
            idxx2 += 1
        idxx1 += 1
    idxy = 0
    for ye in y:
        if np.all(y_data[idx*WINDOW_SIZE+idxy]!=ye):
            print(f'Not match y: {y_data[idx+idxy]}, {ye}')    
            is_match = False
        idxy += 1   
    idx += 1     
    if not is_match:
        print('Not Match')
        break
    
if is_match:
    print('Match')

Match


In [76]:
def make_window(data, window_size):
    window = []
    for i in range(data.shape[0]-window_size):
        element = []
        for j in range(window_size):
            element.append(data[i+j])
        window.append(element)
    return np.array(window)

In [84]:
# 이 코드 맑은 정신으로 다시 확인

x_data_win = make_window(x_data, WINDOW_SIZE)
y_data_win = nd_data[WINDOW_SIZE-1:, TARGET_COL_IDX]

for x, y in ts_train.take(-1):
    is_match = False
    for xe in x:
        idx = 0
        for xt in x_data_win:
            if np.all(xe == xt):
                is_match = True
                print(f'일치하는 속성을 찾았습니다. 인덱스: {idx}')
                print(f'{xe}\n{xt}')
                break
            idx += 1
        if is_match:
            break
    if is_match:
        idy = 0
        for _y1, _y2 in zip(y.numpy(), y_data_win[idx:idx+WINDOW_SIZE]):
            if _y1 != _y2:
                is_match = False
                print(f'레이블이 일치하지 않습니다. 인덱스: ({idx}, {idy}), 값: {_y1}, {_y2}')
                break
            idy += 1
        # if np.all(y.numpy()!=y_data_win[idx*WINDOW_SIZE:idx*WINDOW_SIZE+WINDOW_SIZE]):
        #     is_match = False
        #     print(f'레이블이 일치하지 않습니다. 인덱스: {idx}')
        #     print(y.numpy(), '\n', y_data_win[idx:idx+WINDOW_SIZE])
        #     break
        # else:
        #     print(f'레이블이 일치합니다. 인덱스: {idx}')
        #     print(y.numpy(), '\n', y_data_win[idx:idx+WINDOW_SIZE])
    
    if not is_match:
        print(f'레이블이 일치하지 않습니다. 인덱스: {idx}')
        print(f'{y.numpy()}\n{y_data_win[idx:idx+WINDOW_SIZE]}')
        break
    else:
        print(f'레이블이 일치합니다. 인덱스: {idx}')

if is_match:
    print('Match')        

일치하는 속성을 찾았습니다. 인덱스: 0
[[ 3.35       -7.          1.         22.        ]
 [ 4.04477612  1.0119403   1.         23.        ]
 [ 4.35576923 -1.425       1.          0.        ]
 [ 4.04166667 -0.67833333  1.          1.        ]
 [ 3.7559322   1.92711864  1.          2.        ]
 [ 4.695       1.885       1.          3.        ]
 [ 6.335      -1.23833333  1.          4.        ]
 [ 7.61        3.00166667  1.          5.        ]
 [ 7.245      -0.72333333  1.          6.        ]
 [ 5.39833333 -0.26        1.          7.        ]]
[[ 3.35       -7.          1.         22.        ]
 [ 4.04477612  1.0119403   1.         23.        ]
 [ 4.35576923 -1.425       1.          0.        ]
 [ 4.04166667 -0.67833333  1.          1.        ]
 [ 3.7559322   1.92711864  1.          2.        ]
 [ 4.695       1.885       1.          3.        ]
 [ 6.335      -1.23833333  1.          4.        ]
 [ 7.61        3.00166667  1.          5.        ]
 [ 7.245      -0.72333333  1.          6.        ]
 [ 5.39

In [58]:
a = np.array([ 646.78666667,  404.59666667,  392.86333333,  842.03166667, 1128.27833333, \
 1257.39666667,  812.61666667,  868.8,        1119.99666667, 1473.54833333] )
b = np.array([ 646.78666667,  404.59666667,  392.86333333,  842.03166667, 1128.27833333, \
 1257.39666667,  812.61666667,  868.8,        1119.99666667, 1473.54833333])

In [60]:
if np.all(a != b):
    print('Match')
else:
    print('Not Match')

Not Match
